In [ ]:
import os
import pandas as pd
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.schema import Document
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import SQLChatMessageHistory

# ✅ API Key 設定
os.environ["OPENAI_API_KEY"] = "api key"

# ✅ 載入資料與建構 Chroma 向量庫
persist_dir = "./chroma_db"
text_units_df = pd.read_json("高商合金0406_text_units.json", lines=True)
embeddings_df = pd.read_json("高商合金0406_embeddings.text_unit.text.json", lines=True)
data_df = pd.merge(text_units_df, embeddings_df, on="id")

documents = [
    Document(page_content=row["text"], metadata={"id": row["id"]})
    for _, row in data_df.iterrows()
]

embedding_model = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    persist_directory=persist_dir
)
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# ✅ 提示模板
prompt = ChatPromptTemplate.from_messages([
    ("system", 
     """You are a professional assistant specializing in materials science and engineering.
        Your main role is to help users search, analyze, and summarize materials-related literature and data.
        Always provide factual, source-backed responses and clearly state when information is uncertain or not available.
        When applicable, extract key material properties (e.g., conductivity, thermal stability, mechanical strength) and link them to relevant studies or datasets.
        Avoid making assumptions not supported by data.\n\n{context}"""),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

# ✅ 設定 LLM
llm = ChatOpenAI(model_name="gpt-4o", temperature=0, max_tokens=8192)

# ✅ 建立文件問答鏈
document_chain = create_stuff_documents_chain(llm, prompt)

# ✅ 建立 Retrieval Chain
retrieval_chain = create_retrieval_chain(retriever, document_chain)

# ✅ 定義 SQL 記憶儲存
def get_session_history(session_id):
    return SQLChatMessageHistory(session_id, "sqlite:///./langchain.db")

# ✅ 包裝成具記憶功能的多輪對話鏈
chain_with_history = RunnableWithMessageHistory(
    retrieval_chain,
    get_session_history,
    input_messages_key="input",      # 對應 prompt 中的 human: {input}
    output_messages_key="answer",    # 對應輸出鍵
    history_messages_key="history",  # 對應 prompt 中的 MessagesPlaceholder
)

C:\Users\chen\AppData\Local\Temp\ipykernel_72684\2103740942.py:27: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings()
C:\Users\chen\AppData\Local\Temp\ipykernel_72684\2103740942.py:48: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-4o", temperature=0, max_tokens=8192)


In [3]:
config = {"configurable": {"session_id": "chad-session-0408"}}

response1 = chain_with_history.invoke({"input": "請問在目前的高熵合金應用中，哪些元素組合常見於催化劑？"}, config=config)
print("🤖 回答1：", response1["answer"])

response2 = chain_with_history.invoke({"input": "哪一組的表現比較好？"}, config=config)
print("🤖 回答2：", response2["answer"])

🤖 回答1： 高熵合金（High-Entropy Alloys, HEAs）在催化劑應用中因其獨特的組成和結構特性而受到廣泛關注。這些合金通常由多種元素組成，且每種元素的比例相近。以下是一些常見於催化劑應用中的高熵合金元素組合：

1. **Ni-Co-Fe-Cr-Mn**：這組合常用於氧還原反應（ORR）和氫演化反應（HER），因為這些元素能提供良好的催化活性和穩定性。

2. **Pt-Pd-Rh-Ru-Ir**：這些貴金屬組合在催化劑中非常常見，特別是在汽車尾氣處理和燃料電池中，因為它們具有優異的催化性能。

3. **Cu-Ni-Co-Cr-Fe**：這組合在甲烷重整和其他碳氫化合物轉化反應中表現出色，因為它們能有效地促進反應並提高選擇性。

4. **V-Nb-Ta-Mo-W**：這些過渡金屬組合在氨合成和其他高溫催化反應中被研究，因為它們具有高熔點和良好的化學穩定性。

這些元素組合的選擇通常基於它們的電子結構、原子半徑、熔點和化學穩定性等因素，以優化催化性能和耐久性。具體的應用和性能還需根據實驗數據和理論計算進行進一步驗證。
🤖 回答2： 在高熵合金作為催化劑的應用中，哪一組元素組合表現較好通常取決於特定的催化反應和應用環境。以下是一些考量因素：

1. **反應類型**：不同的反應對催化劑的要求不同。例如，氧還原反應（ORR）和氫演化反應（HER）可能需要不同的活性位點和電子結構。

2. **操作條件**：高溫、高壓或酸鹼環境等操作條件會影響合金的穩定性和活性。例如，V-Nb-Ta-Mo-W組合可能在高溫反應中表現較好。

3. **成本和可持續性**：貴金屬如Pt-Pd-Rh-Ru-Ir組合雖然性能優異，但成本高昂且資源有限。因此，在某些應用中，可能會選擇成本較低的組合如Ni-Co-Fe-Cr-Mn。

4. **穩定性和耐久性**：在長時間運行中，催化劑的穩定性和抗中毒能力是關鍵因素。某些組合可能在這方面表現更佳。

總的來說，沒有一個單一的高熵合金組合能在所有催化應用中表現最佳。研究者通常需要根據具體的應用需求和實驗結果來選擇和優化合金組合。實驗數據和理論計算是評估和選擇合適合金的關鍵。


In [ ]:
config = {"configurable": {"session_id": "chad-session-0408"}}

response1 = chain_with_history.invoke({"input": "請問在目前的高熵合金應用中，哪些元素組合常見於催化劑？"}, config=config)
print("🤖 回答1：", response1["answer"])

response2 = chain_with_history.invoke({"input": "哪一組的表現比較好？"}, config=config)
print("🤖 回答2：", response2["answer"])


c:\Users\chen\anaconda3\envs\langchainragas\lib\site-packages\langchain_core\runnables\history.py:608: LangChainDeprecationWarning: `connection_string` was deprecated in LangChain 0.2.2 and will be removed in 1.0. Use connection instead.
  message_history = self.get_session_history(


🤖 回答1： 高熵合金（High-Entropy Alloys, HEAs）在催化劑應用中，由於其獨特的多元組成和優異的物理化學性質，受到廣泛關注。常見的高熵合金催化劑元素組合通常包括過渡金屬元素，如：

1. 鎳（Ni）
2. 鈷（Co）
3. 鐵（Fe）
4. 銅（Cu）
5. 鉻（Cr）
6. 鉬（Mo）
7. 鈦（Ti）

這些元素可以組合成多元合金，提供優異的催化性能，如高活性、穩定性和抗中毒能力。具體的元素組合會根據目標反應的需求而有所不同。
🤖 回答2： 高熵合金在催化劑應用中的表現取決於具體的反應類型和操作條件，因此很難單獨指出哪一組元素組合的表現最好。然而，一些研究表明，某些特定組合在特定反應中表現出色。例如：

1. **氧還原反應（ORR）**：Ni-Co-Fe-Cr-Mn高熵合金在燃料電池中的氧還原反應中表現良好，因為這些元素提供了良好的電子導電性和穩定性。

2. **氫演化反應（HER）**：Ni-Fe-Mo-Cr-Ti合金在電解水制氫中顯示出優異的催化活性和耐久性。

3. **甲烷重整**：Ni-Co-Cu-Fe合金在甲烷重整反應中被用作有效的催化劑，因為它們能夠提供高的催化活性和抗積碳能力。

具體哪一組表現最好需要根據實驗數據和應用需求來確定，並且通常需要進行進一步的優化和調整以達到最佳性能。


<h1>問題1:請問在目前的高熵合金應用中，哪些元素組合常見於催化劑？</h1> </br>
<h1>問題2:哪一組的表現比較好？</h1>

<H1>使用MSgraph切分PDF當成輸入資料</H1>
🤖 回答1： 高熵合金（High-Entropy Alloys, HEAs）在催化劑應用中，由於其獨特的多元組成和優異的物理化學性質，受到廣泛關注。常見的高熵合金催化劑元素組合通常包括過渡金屬元素，如：

1. 鎳（Ni）
2. 鈷（Co）
3. 鐵（Fe）
4. 銅（Cu）
5. 鉻（Cr）
6. 鉬（Mo）
7. 鈦（Ti）

這些元素可以組合成多元合金，提供優異的催化性能，如高活性、穩定性和抗中毒能力。具體的元素組合會根據目標反應的需求而有所不同。</br>
🤖 回答2： 高熵合金在催化劑應用中的表現取決於具體的反應類型和操作條件，因此很難單獨指出哪一組元素組合的表現最好。然而，一些研究表明，某些特定組合在特定反應中表現出色。例如：

1. **氧還原反應（ORR）**：Ni-Co-Fe-Cr-Mn高熵合金在燃料電池中的氧還原反應中表現良好，因為這些元素提供了良好的電子導電性和穩定性。

2. **氫演化反應（HER）**：Ni-Fe-Mo-Cr-Ti合金在電解水制氫中顯示出優異的催化活性和耐久性。

3. **甲烷重整**：Ni-Co-Cu-Fe合金在甲烷重整反應中被用作有效的催化劑，因為它們能夠提供高的催化活性和抗積碳能力。

具體哪一組表現最好需要根據實驗數據和應用需求來確定，並且通常需要進行進一步的優化和調整以達到最佳性能。

<H1>使用Native.ipynb，純PDF當成輸入資料</H1>
<text>
🤖 回答1： 在高熵合金（HEAs）應用於催化劑的研究中，常見的元素組合包括過渡金屬元素，如鎳（Ni）、鈷（Co）、鐵（Fe）、銅（Cu）、鉻（Cr）、錳（Mn）等。這些元素的組合可以形成具有多樣性和穩定性的合金系統，這些特性對於催化劑的性能提升非常有利。此外，某些研究也會加入貴金屬元素如鉑（Pt）、鈀（Pd）等，以進一步提高催化活性和選擇性。這些高熵合金催化劑在多相催化、電催化和光催化等領域展現出潛力。</br>
🤖 回答2： 高熵合金在催化劑應用中的表現取決於具體的反應和應用場景，因此很難單純地說哪一組元素的表現最好。然而，一些研究表明，包含貴金屬如鉑（Pt）和鈀（Pd）的高熵合金在某些催化反應中表現出色，特別是在需要高催化活性和選擇性的反應中。

另一方面，完全由過渡金屬組成的高熵合金（如Ni、Co、Fe、Cu、Cr、Mn等）在成本和穩定性方面具有優勢，並且在某些工業應用中也能提供良好的催化性能。

因此，選擇哪一組元素組合取決於具體的應用需求、成本考量以及所需的催化性能。研究人員通常會根據這些因素進行優化和調整，以達到最佳的催化效果。
</text>